In [13]:
from bs4 import BeautifulSoup
from parse_utils import *
from url_utils import get_urls_from_afisha_and_online_cinema
import requests


Получаем все ссылки на фильмы c афишы и онлайн кинотеатра

In [14]:


    
base_url_afisha = 'https://afisha.tut.by/film/'
base_url_online_cinema='https://afisha.tut.by/online-cinema/'
urlsOfFilms=get_urls_from_afisha_and_online_cinema(base_url_afisha,base_url_online_cinema)
print("Number of films "+str(len(urlsOfFilms)))

Number of films 87
25
50
75
100
125
150
175
200
225
250
275
300
325
350
375
400
425
450
475
500
525
550
575
600
625
650
675
700
725
750
775
800
825
850
875
900
925
950
975
1000
1025
1050
1075
1100
1125
1150
1175
1200
1225
1250
1275
1300
1325
1350
1375
1400
1425
1450
1475
1500
1525
1550
1575
1600
1625
1650
1675
1700
1725
1750
1775
1800
1825
1850
1875
1900
1925
1950
1975
2000
2025
2050
2075
2100
2125
2150
2175
2200
2225
2250
2275
2300
2325
2350
2375
2400
2425
2450
2475
2500
2525
2550
2575
2600
2625
2650
2675
2700
2725
2750
2775
2800
2825
2850
2875
2900
2925
2950
2975
3000
3025
3050
3075
3100
3125
3150
3175
3190
2 3190



Пишем функцию для парсинга страницы

In [15]:
def parse_film_page(url):
    response_of_film_page = requests.get(url)
    parsed_film_page = BeautifulSoup(response_of_film_page.text, 'html.parser')
    data={'url':url,'name':get_film_name(parsed_film_page)} #set name and url of film
    data['is_premiere']=is_premiere(parsed_film_page) #is this film premiere
    data['age_rank']=get_film_age_rank(parsed_film_page) # age rank  (16+ and etc.)
    data['native_film_name']=get_native_film_name(parsed_film_page) # name of film in original
    data['poster_url']=get_poster_url(parsed_film_page) #url to film poster
    data['genres']=get_genre(parsed_film_page) # genres of film
    data['year']=get_year_of_film(parsed_film_page) #year of creation
    data['duration']=get_duration(parsed_film_page) #duration
    data['countries']=get_country(parsed_film_page) #countries 
    data['showing_end_date']=get_showing_end_date(parsed_film_page) #end date of showing in theaters
    data['desription']=get_description(parsed_film_page) #text description of film
    data['ratings']=get_ratings(parsed_film_page) # ratings
    data['directors']=get_director(parsed_film_page) #directors of film and link for additional director information
    data['actors']=get_actors(parsed_film_page) #actors  and link for additional actors information
    data['film_videos']=get_film_videos(parsed_film_page) #urls to videos of film (trailer and etc.)
    data['film_images']=get_film_images(parsed_film_page) #urls to images of film 
    return data


Пишем функцию для парсинга страниц

In [16]:
def parse_all_pages(urls):
    parsed_pages={}
    print("***************************START*****************************")
    count=len(urls)
    for index,url in enumerate(urls):
        print("Scrappring url "+url+" ( "+index+" / "+count+" )")
        film_data=parse_film_page(url)
        parsed_pages[film_data['name']]=film_data
    print("***************************END*****************************")
    return parsed_pages


Парсим страницы

In [17]:
data=parse_all_pages(urlsOfFilms)
print("Prepared data for "+str(len(data))+" films")

https://afisha.tut.by/film/theatrehd-veer-ledi-uindermir-spektakl/
https://afisha.tut.by/film/promar/
https://afisha.tut.by/film/gemini/
https://afisha.tut.by/film/everybody-in-the-place/
https://afisha.tut.by/film/dozhdlivyy-den-v-nyu-yorke/
https://afisha.tut.by/film/muzey-prado-kollekciya-chudes/
https://afisha.tut.by/film/33-slova-o-dizayne/
https://afisha.tut.by/film/chas-letapisu-anatoliy-alay/
https://afisha.tut.by/film/eto-ne-navsegda/
https://afisha.tut.by/film/materinskiy_instinkt/
https://afisha.tut.by/film/depeche_mode_spirits_in_the_forest/
https://afisha.tut.by/film/ty-umeesh-khranit-sekrety/
https://afisha.tut.by/film/noch_kino_v_moskve_ghost/
https://afisha.tut.by/film/theatrehd-mudrye-deti/
https://afisha.tut.by/film/studiya-54/
https://afisha.tut.by/film/korol-lev/
https://afisha.tut.by/film/viy-2-tayna-pechati-drakona/
https://afisha.tut.by/film/bozhe-moy/
https://afisha.tut.by/film/theatrehd-kanova/
https://afisha.tut.by/film/shchegol/
https://afisha.tut.by/film/tay